**Created by Sanskar Hasija**

**🤖H2O.ai AutoML Classification - Titanic**

**15 NOVEMBER 2021**


# <center> 🤖H2O.AI AUTOML CLASSIFICATION - TITANIC</center>
## <center>If you find this notebook useful, support with an upvote👍</center>

# Imports

In [1]:
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

# Initializing H20 Cluster

In [2]:
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnuccriv5
  JVM stdout: /tmp/tmpnuccriv5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpnuccriv5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_unknownUser_wicfi4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.250 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


# Data Loading and Preprocessing

In [3]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv('../input/titanic/test.csv')
subs = pd.read_csv('../input/titanic/gender_submission.csv')

drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)

str_list = [] 
num_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)
        
train = pd.get_dummies(train, columns=str_list)
test = pd.get_dummies(test, columns=str_list)

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
train.describe()

Rows:891
Cols:9




,Survived,Pclass,Age,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
type,int,int,real,real,int,int,int,int,int
mins,0.0,1.0,0.42,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.3838383838383838,2.3086419753086447,29.699117647058795,32.20420796857465,0.35241301907968575,0.6475869809203143,0.18855218855218855,0.08641975308641975,0.7250280583613917
maxs,1.0,3.0,80.0,512.3292,1.0,1.0,1.0,1.0,1.0
sigma,0.4865924542648575,0.8360712409770491,13.002015226002891,49.69342859718089,0.4779900708960982,0.4779900708960982,0.3913721645054733,0.28114069214170423,0.44675091003414663
zeros,549,0,0,15,577,314,723,814,245
missing,0,0,0,0,0,0,0,0,0
0,0.0,3.0,22.0,7.25,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1.0,3.0,26.0,7.925,1.0,0.0,0.0,0.0,1.0


# AutoML

In [5]:
MAX_RUNTIME = 1800 #30 mins
RANDOM_SEED = 12 

y = "Survived"
train[y] = train[y].asfactor()
x = train.columns
x.remove(y)

In [6]:
aml = H2OAutoML(seed = RANDOM_SEED , max_runtime_secs=MAX_RUNTIME)
aml.train(x = x, y = y, training_frame = train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_1_20211115_120125

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.06858223207426449
RMSE: 0.26188209574971805
LogLoss: 0.26306487742561363
Null degrees of freedom: 890
Residual degrees of freedom: 879
Null deviance: 1186.6551368246774
Residual deviance: 468.7816115724435
AIC: 492.7816115724435
AUC: 0.9870471564460636
AUCPR: 0.9808612188284009
Gini: 0.9740943128921271

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4019092658221778: 


,,0,1,Error,Rate
0,0,528.0,21.0,0.0383,(21.0/549.0)
1,1,32.0,310.0,0.0936,(32.0/342.0)
2,Total,560.0,331.0,0.0595,(53.0/891.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.401909,0.921248,199.0
1,max f2,0.261136,0.949438,243.0
2,max f0point5,0.457913,0.943396,186.0
3,max accuracy,0.450213,0.940516,188.0
4,max precision,0.996973,1.000000,0.0
5,max recall,0.163277,1.000000,306.0
6,max specificity,0.996973,1.000000,0.0
7,max absolute_mcc,0.450213,0.874196,188.0
8,max min_per_class_accuracy,0.334449,0.932605,214.0
9,max mean_per_class_accuracy,0.326001,0.936703,218.0



Gains/Lift Table: Avg response rate: 38.38 %, avg score: 39.04 %


...

,,0,1,Error,Rate
0,0,484.0,65.0,0.1184,(65.0/549.0)
1,1,82.0,260.0,0.2398,(82.0/342.0)
2,Total,566.0,325.0,0.165,(147.0/891.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.415380,0.779610,189.0
1,max f2,0.216261,0.810308,281.0
2,max f0point5,0.545865,0.830871,148.0
3,max accuracy,0.537674,0.838384,151.0
4,max precision,0.987294,1.000000,0.0
5,max recall,0.024260,1.000000,398.0
6,max specificity,0.987294,1.000000,0.0
7,max absolute_mcc,0.537674,0.656228,151.0
8,max min_per_class_accuracy,0.338419,0.815789,219.0
9,max mean_per_class_accuracy,0.397908,0.820942,196.0



Gains/Lift Table: Avg response rate: 38.38 %, avg score: 38.38 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010101,0.946201,2.605263,2.605263,1.000000,0.961922,1.000000,0.961922,0.026316,0.026316,160.526316,160.526316,0.026316
1,2,0.020202,0.938237,2.605263,2.605263,1.000000,0.944075,1.000000,0.952999,0.026316,0.052632,160.526316,160.526316,0.052632
2,3,0.030303,0.929784,2.605263,2.605263,1.000000,0.934956,1.000000,0.946984,0.026316,0.078947,160.526316,160.526316,0.078947
3,4,0.040404,0.924440,2.605263,2.605263,1.000000,0.926808,1.000000,0.941940,0.026316,0.105263,160.526316,160.526316,0.105263
4,5,0.050505,0.918440,2.605263,2.605263,1.000000,0.922454,1.000000,0.938043,0.026316,0.131579,160.526316,160.526316,0.131579
5,6,0.101010,0.863764,2.431579,2.518421,0.933333,0.890952,0.966667,0.914498,0.122807,0.254386,143.157895,151.842105,0.248921
6,7,0.150393,0.779578,2.368421,2.469167,0.909091,0.826233,0.947761,0.885515,0.116959,0.371345,136.842105,146.916732,0.358595
7,8,0.200898,0.700408,2.373684,2.445163,0.911111,0.743846,0.938547,0.849900,0.119883,0.491228,137.368421,144.516319,0.471192
8,9,0.300786,0.510627,1.873448,2.255302,0.719101,0.608158,0.865672,0.769620,0.187135,0.678363,87.344766,125.530244,0.612789
9,10,0.400673,0.371816,1.141632,1.977665,0.438202,0.434717,0.759104,0.686129,0.114035,0.792398,14.163217,97.766475,0.635749


# Results Leaderboard

### Top 10 models

In [7]:
lb = aml.leaderboard
lb.head(10)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_1_20211115_120125,0.877273,0.416695,0.859902,0.179082,0.357514,0.127816
StackedEnsemble_Best1000_1_AutoML_1_20211115_120125,0.876067,0.416507,0.860067,0.177548,0.357198,0.12759
StackedEnsemble_AllModels_5_AutoML_1_20211115_120125,0.875731,0.522331,0.830589,0.173425,0.370539,0.137299
StackedEnsemble_AllModels_7_AutoML_1_20211115_120125,0.875691,0.414485,0.857008,0.176469,0.356615,0.127174
StackedEnsemble_AllModels_4_AutoML_1_20211115_120125,0.874341,0.419161,0.856758,0.182389,0.359398,0.129167
StackedEnsemble_BestOfFamily_4_AutoML_1_20211115_120125,0.873198,0.400306,0.857021,0.175055,0.35023,0.122661
StackedEnsemble_BestOfFamily_8_AutoML_1_20211115_120125,0.873188,0.403522,0.854494,0.168416,0.35188,0.12382
GBM_5_AutoML_1_20211115_120125,0.873143,0.40102,0.857888,0.171556,0.35114,0.1233
StackedEnsemble_BestOfFamily_5_AutoML_1_20211115_120125,0.872922,0.402472,0.854754,0.165924,0.351079,0.123256
StackedEnsemble_AllModels_2_AutoML_1_20211115_120125,0.872197,0.40166,0.859177,0.171724,0.350714,0.123


### Submission

In [8]:
preds = aml.leader.predict(test)
sub = preds[0].as_data_frame()  ## Converting to dataframe
subs['Survived'] = sub
subs.to_csv('h20.ai-automl.csv',index = False)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
